In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
files_l = []
files_r = []
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if 'Left' in filename:
            files_l.append(os.path.join(dirname, filename))
        else:
            files_r.append(os.path.join(dirname, filename))

In [2]:
if ( len(files_r)!= len(files_l)):
    print("Incomplete Conversation Detected")
else:
    files_r.sort()
    files_l.sort()

In [3]:
import json

output_data = {}
conv_id = 70000

for left, right in zip(files_l, files_r):
    with open(left, 'r') as f:
        client = json.load(f)
    with open(right, 'r') as f:
        agent = json.load(f)

    client_messages = client['value']['segments']
    agent_messages = agent['value']['segments']

    messages = []
    turn = None
    message = []

    mID = 1
    i = j = 0
    try:
        for _ in range(len(client_messages) + len(agent_messages)):
            client_id = int(client_messages[i]['segmentId']) if i < len(client_messages) else float('inf')
            agent_id = int(agent_messages[j]['segmentId']) if j < len(agent_messages) else float('inf')
    
            curr = 0 if client_id < agent_id else 1
    
            if curr == 0:
                content = client_messages[i].get('transcriptionData', {}).get('content', '').strip()
                i += 1
            else:
                content = agent_messages[j].get('transcriptionData', {}).get('content', '').strip()
                j += 1
    
            if content:
                message.append(content)
    
            if turn != curr:
                if message:
                    messages.append({
                        'from': 'Client' if curr == 0 else 'Agent',
                        'message_id': mID,
                        'message': ' '.join(message)
                    })
                    message = []
                    mID += 1
                turn = curr
    
        output_data[conv_id] = messages
        conv_id += 1
    except:
        print(left,right,i,j)
print(f"Extracted Conversations: {len(output_data)}")


Extracted Conversations: 16


In [4]:
#with open('CallCentreTranscripts.json','w') as f:
#    json.dump(output_data,f,indent=2)